In [207]:
import cbpro
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import arrow
from requests.auth import AuthBase
import plotly.graph_objs as go

In [3]:
api_url = 'https://api.pro.coinbase.com/'
auth = cbpro.AuthenticatedClient(username,
                            secretkey,
                            key,api_url)

In [1]:
d_end = arrow.get('2021-01-25')
d_end_date = d_end.format('YYYY-MM-DD')
d_start = arrow.get('2021-01-15')
d_start_date = d_start.format('YYYY-MM-DD')
d_limit = arrow.get('2016-01-01')
d_limit_date = arrow.get('2016-01-01').format('YYYY-MM-DD')
full_data = []
#x = 3600
while d_start.timestamp > d_limit.timestamp:    

    x = 3600*24
    data = auth.get_product_historic_rates('ETH-USD',start=d_start_date,end=d_end_date,granularity=x)
    #time.sleep(2)
    if type(data) != dict:
        full_data.append(data)
        #print(full_data)
        d_start = arrow.get(full_data[-1][-1][0]-(230*x))
        #print(data[-1][0])
        d_start_date = arrow.Arrow.fromtimestamp(full_data[-1][-1][0]-(230*x)).format('YYYY-MM-DD')
        d_end_date = arrow.Arrow.fromtimestamp(full_data[-1][-1][0]).format('YYYY-MM-DD')
        #print(d_start_date,'if')
    else:
        d_start = arrow.get(full_data[-1][-1][0]-(230*x))
        #print(data[-2][-1][0])
        d_start_date = arrow.Arrow.fromtimestamp(full_data[-1][-1][0]-(230*x)).format('YYYY-MM-DD')
        d_end_date = arrow.Arrow.fromtimestamp(full_data[-1][-1][0]).format('YYYY-MM-DD')
        #print(d_start_date,'else')
    d_start = d_start
    d_start_date = d_start_date
    d_end_date = d_end_date
    
    

NameError: name 'arrow' is not defined

In [258]:
arrow.Arrow.fromtimestamp(1537315200).format('YYYY-MM-DD HH:mm')

'2018-09-19 01:00'

In [259]:
#As the outcome from the api request is a list of a list with the loop, I am creating just a list of the total
# requested data
clean = []
for i in full_data:
    for j in i:
        clean.append(j)

In [260]:
pd.DataFrame(clean,columns=['time','low','high','open','close','volume']).to_csv('ETHDaily.csv',index=False)

In [261]:
df = pd.read_csv('ETHDaily.csv')
#df.drop('Unnamed: 0',axis=1,inplace=True)

In [262]:
#df['Date'] = [arrow.Arrow.fromtimestamp(date).format('YYYY-MM-DD HH:mm') for date in df['time']]

In [263]:
df.drop_duplicates(inplace=True)

In [264]:
df.head()

,time,low,high,open,close,volume
0,1611532800,1293.65,1477.30,1394.04,1317.97,627535.527937
1,1611446400,1221.66,1401.92,1234.47,1394.00,434746.405089
2,1611360000,1196.74,1272.00,1233.42,1234.55,269855.020808
3,1611273600,1039.62,1275.99,1110.61,1233.60,758560.995474
4,1611187200,1081.00,1390.00,1377.15,1110.62,789194.344680


In [265]:
df['check'] = df.time.diff(periods=1)

In [268]:
#check if there is any daily, hourly or 15min data missing where the timestamp diff 
#is more than the wanted interval (daily,hourly,15min..etc)
df[df['check'] != -3600*24]

,time,low,high,open,close,volume,check
0,1611532800,1293.65,1477.30,1394.04,1317.97,627535.527937,NaN
1717,1463702400,13.71,14.82,14.82,14.17,254.166617,-259200.0


In [220]:
#df['new_time'] = [(df.time[0]-(900*i)) for i in range(0,len(df.time),1)]

In [277]:
#create the correct timestamp part1
timestamp = int((df['time'][0] - df['time'][-1:])/(3600*24))

In [278]:
df['time'][-1:]

1719    1463529600
Name: time, dtype: int64

In [279]:
#create the correct timestamp part2
timestamps = [df.time[0]-(3600*24*i) for i in range(timestamp+1)]

In [280]:
#create timestamp dataframe
timestamps = pd.DataFrame(np.array(timestamps),columns=['time'])

In [282]:
#merge on new timestamp with previous dataframe to be sure it all talies up
new_df = pd.merge(df,timestamps,how='right',on=['time']).reset_index(drop=True)

In [283]:
new_df = new_df.fillna(method='bfill',axis=0)

In [284]:
new_df[new_df.low.isnull() == True]

,time,low,high,open,close,volume,check


In [285]:
new_df.shape

(1714, 7)

In [286]:
new_df.drop(['check'],axis=1,inplace=True)

In [287]:
new_df.columns

Index(['time', 'low', 'high', 'open', 'close', 'volume'], dtype='object')

In [288]:
Cols = ['Time','Low','High','Open','Close','Volume']
new_df.columns = Cols

In [289]:
new_df['Date'] = [arrow.Arrow.fromtimestamp(date).format('YYYY-MM-DD HH:mm') for date in new_df['Time']]

In [290]:
new_df = new_df.drop_duplicates().set_index('Date')
new_df.shape

(1714, 6)

In [291]:
new_df.head()

,Time,Low,High,Open,Close,Volume
Date,,,,,,
2021-01-25 00:00,1611532800,1293.65,1477.30,1394.04,1317.97,627535.527937
2021-01-24 00:00,1611446400,1221.66,1401.92,1234.47,1394.00,434746.405089
2021-01-23 00:00,1611360000,1196.74,1272.00,1233.42,1234.55,269855.020808
2021-01-22 00:00,1611273600,1039.62,1275.99,1110.61,1233.60,758560.995474
2021-01-21 00:00,1611187200,1081.00,1390.00,1377.15,1110.62,789194.344680


In [292]:
new_df.drop(['Time'],axis=1,inplace=True)

In [293]:
#save csv 
new_df.to_csv('ETHDaily.csv')